# Compute PCA within the 2L chrom

In [ ]:
!pip install -q malariagen_data
!pip install -q scikit-allel
!pip install -q petl

     |████████████████████████████████| 100 kB 9.2 MB/s 
     |████████████████████████████████| 301 kB 69.0 MB/s 
     |████████████████████████████████| 3.6 MB 54.2 MB/s 
     |████████████████████████████████| 185 kB 53.3 MB/s 
     |████████████████████████████████| 3.7 MB 67.0 MB/s 
     |████████████████████████████████| 9.9 MB 51.0 MB/s 
     |████████████████████████████████| 5.7 MB 64.8 MB/s 
     |████████████████████████████████| 3.0 MB 58.7 MB/s 
     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 1.6 MB 66.9 MB/s 
     |████████████████████████████████| 6.6 MB 52.8 MB/s 
     |████████████████████████████████| 408 kB 26.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


### importing necessary package

In [ ]:
import allel
import malariagen_data
import numpy as np
import pandas as pd
import dask
import dask.array as da
# silence some dask warnings
dask.config.set(**{'array.slicing.split_large_chunks': True})
from dask.diagnostics.progress import ProgressBar

In [ ]:
import random
#import functools
#import petl as ptl
#import itertools
import scipy
from pathlib import Path
from collections import Counter

In [ ]:
# plotting setup
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.gridspec import GridSpec
import matplotlib_venn as venn
import seaborn as sns

In [20]:
#Mounting Google Drive
import os
from google.colab import drive
drive.mount("drive")

# make dir 
results_dir = "drive/MyDrive/Genomic/"
os.makedirs(results_dir, exist_ok=True)

Mounted at drive


Importing malariagen data set    
Let's select the the sample set from 

In [ ]:
ag3 = malariagen_data.Ag3("gs://vo_agam_release/", pre=True)
#ag3
sets = ["1191-VO-MULTI-OLOUGHLIN-VMF00106", "1191-VO-MULTI-OLOUGHLIN-VMF00140", 
             "AG1000G-BF-A", "AG1000G-BF-B", "AG1000G-BF-C"]
df_samples = ag3.sample_metadata(sample_sets=sets)#.set_index("sample_id")
#bf_samples = df_samples.query('country == "Burkina Faso"')
#bf_samples.groupby(['location', 'aim_species']).size()

#To access to the snp genotypes within the 2L chromosomes
chrom2L_gt = ag3.snp_calls(region="2L", sample_sets=sets)


Load sample metadata:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
# define population 
bf_samples = df_samples.query('country == "Burkina Faso" and year > 2004')
s1, s2, pop_labels,pop_colors, = [], [], {}, {}
for iso, species in zip(bf_samples.location, bf_samples.aim_species):
    s1.append(iso[:4]+'_'+species[:3])
bf_samples.insert(4, 'population', s1)
bf_samples_sel = pd.concat([bf_samples, bf_samples])
pop_ids = list(bf_samples.population.unique())

#pop_id
for idx, species in zip(bf_samples.location, bf_samples.aim_species):
  w = idx.split(' ')[0][:14]
  s2.append(w+' $An.'+species)
val = list(Counter(s2).keys())
for idx, vl in zip(pop_ids, val):
    pop_labels[idx]=vl
pop_labels

#pop_labels & pop_colors
colored = sns.color_palette("husl", len(pop_ids))
for i in range(len(pop_ids)):
    pop_colors[pop_ids[i]] = colored[i]

pop_color = {'Bana_col':'#7FFFD4', 'Pala_col':'#008000', 'Sour_col':'#9ACD32',
             'Bana_gam':'#FF00FF', 'Pala_gam':'#C20078', 'Sour_gam':'#9A0EEA',
             'Bana_ara':'#FFA500', 'Pala_ara':'#DAA520', 'Sour_ara':'#FF6347',
             'Bana_int':'#580F41', 'Pala_int':'#01153E'
             }
#pop_cohort
coh_pop = dict([(f"{p}", list(df.index)) for (p), df in bf_samples.reset_index().groupby(['population'])])
ls1 = ['Pala_gam', 'Sour_gam', 'Sour_col', 'Bana_gam', 'Pala_ara', 'Bana_col']
pop_idx = []
for pop in ls1:
  pop_idx = pop_idx + coh_pop[pop]
len(pop_idx)

#remove some populations 
pop_rm = ['Bana_ara', 'Bana_int', 'Sour_ara', 'Mono_gam', 'Pala_col']
del s1, s2, val, colored

In [ ]:
bf_samples.population.unique()

array(['Pala_gam', 'Sour_gam', 'Sour_col', 'Bana_gam', 'Pala_ara',
       'Bana_col', 'Pala_int', 'Sour_ara', 'Bana_int', 'Pala_col',
       'Bana_ara'], dtype=object)

In [9]:
#To the genomics dataset within the Target region and bf samples 
bf_snps = chrom2L_gt.sel(samples=df_samples.eval('country=="Burkina Faso" and year>2004'))

# Take some SNP in the 2R chromosomes 
bf_snps_pos = allel.SortedIndex(bf_snps['variant_position'].values)

# To filter the SNP dataset and warp the dataset to GT array 
filt = 'gamb_colu_arab' 
filt_val = bf_snps[f"variant_filter_pass_{filt}"].values 
gt_filtered = allel.GenotypeDaskArray(bf_snps["call_genotype"][filt_val].data)

# compute allele counts
with ProgressBar():
  ac = gt_filtered.take(pop_idx, axis=1).count_alleles(max_allele=3).compute()
#ac_pop = gt_filtered.count_alleles_subpops(coh_pop)

[########################################] | 100% Completed |  1hr 12min 22.7s


#PCA

In [10]:
#filter - to remove missing data
filter_ac = (ac.max_allele() == 1) & (ac[:, :2].min(axis=1) > 1)
gt_rm_miss = gt_filtered.compress(filter_ac, axis=0)
geno_alt = gt_rm_miss.to_n_alt()

In [11]:
#randomly select snps 
n=300000   
rand_x = np.random.choice(geno_alt.shape[0], n, replace=False)
rand_x.sort()
gt_random = gt_rm_miss.take(rand_x, axis=0).to_n_alt()

In [12]:
#define PCA path 
pca_data = Path('pca_data.npz')
pca_data

PosixPath('pca_data.npz')

In [13]:
if pca_data.is_file():
    # file exist
    pca = np.load('pca_data.npz')
    coords = pca['coords']
    loadings = pca['loadings']
    variance = pca['variance']
else:
    print('processing')
    coords, model = allel.pca(gt_random, n_components=20, scaler='patterson')

processing


In [23]:
## save PCA data 
np.save('drive/MyDrive/Genomic/PCA1/2L_coords', coords)
np.save('drive/MyDrive/Genomic/PCA1/2L_model', model)

def load_coords(file='coords.npy'):
  return np.load(f'drive/MyDrive/Genomic/PCA1/{file}')

def load_model(file='model.npy'):
  return np.load(f'drive/MyDrive/Genomic/PCA1/{file}', allow_pickle='TRUE').item()

In [ ]:
fig = plt.figure(figsize=(20, 6))
ax = fig.add_subplot(121)
if 'variance' in globals():
    ax.plot(variance, 'go')
else:
    ax.plot(model.explained_variance_ratio_, 'go')
ax.set_xlabel("principal component")
ax.set_ylabel("variance explained")
plt.xticks(np.arange(0,20, 1));

In [ ]:
def marker_style (pop_ids):
  marker = {}
  for pop in pop_ids:
    if pop.split('_')[1]=='gam':
      marker[pop] = 'o'
    elif pop.split('_')[1]=='col':
      marker[pop] = 'D'
    elif pop.split('_')[1]=='ara':
      marker[pop] = 'v'
    elif pop.split('_')[1]=='int':
      marker[pop] = '*'
    else:
      marker[pop] = 'h'
  return marker

In [ ]:
def plot_pca_coords(coords, model, pc1, pc2, ax, sample_population):
    sns.despine(ax=ax, offset=5)
    x = coords[:, pc1]
    y = coords[:, pc2]
    for pop in pop_idx:
        flt = (sample_population == pop)
        ax.plot(x[flt], y[flt], marker=marker_style(pop_ids)[pop], linestyle=' ', color=pop_color[pop], 
                label=pop, markersize=6, mec='k', mew=0.4)
    ax.set_xlabel('PC%s (%.1f%%)' % (pc1+1, model[pc1]*100))
    ax.set_ylabel('PC%s (%.1f%%)' % (pc2+1, model[pc2]*100))
    

In [ ]:
def fig_pca(coords, model, title, sample_population, fig_name=None):
    # plot coords for PCs 1 vs 2, 3 vs 4
    fig = plt.figure(figsize=(16, 10))
    ax = fig.add_subplot(231)
    plot_pca_coords(coords, model, 0, 1, ax, sample_population)

    ax = fig.add_subplot(232)
    plot_pca_coords(coords, model, 2, 3, ax, sample_population)
    
    ax = fig.add_subplot(233)
    plot_pca_coords(coords, model, 4, 5, ax, sample_population)
    
    ax = fig.add_subplot(234)
    plot_pca_coords(coords, model, 6, 7, ax, sample_population)
    
    ax = fig.add_subplot(235)
    plot_pca_coords(coords, model, 8, 9, ax, sample_population)
    
    ax = fig.add_subplot(236)
    plot_pca_coords(coords, model, 10, 11, ax, sample_population)

    ax.legend(bbox_to_anchor=(1, 1), loc='upper left')
    
    fig.suptitle(title, y=1.02)
    fig.tight_layout()
    
    if fig_name:
      fig.savefig(f"drive/MyDrive/Genomic/PCA/{fig_name}.png", dpi=300, bbox_inches='tight')


In [ ]:
if 'variance' in globals():
    fig_pca(coords, variance ,'PCA - 2L', bf_samples.population.values, fig_name='2L_PCA')
else:
    fig_pca(coords, model.explained_variance_ratio_ ,'PCA - 2L', bf_samples.population.values, fig_name='2L_PCA')

In [ ]:
def fig_pca1_2(coords, model, title, sample_population, fig_name=None):
    # plot coords for PCs 1 vs 2
    fig = plt.figure(figsize=(8, 5))
    ax = fig.add_subplot()
    plot_pca_coords(coords, model, 0, 1, ax, sample_population)

    ax.legend(bbox_to_anchor=(1.1, 1), loc='upper left')
    
    fig.suptitle(title, y=1.02)
    fig.tight_layout()

    if fig_name:
      fig.savefig(f"drive/MyDrive/Genomic/PCA/{fig_name}.png", dpi=300, bbox_inches='tight')


In [ ]:
if 'variance' in globals():
    fig_pca1_2(coords, variance ,'PCA1_2 - 2L', bf_samples.population.values, fig_name='2L_PCA12')
else:
    fig_pca1_2(coords, model.explained_variance_ratio_ ,'PCA1_2 - 2L', bf_samples.population.values, fig_name='2L_PCA12')